In [1]:
import torch
from torch import nn
import torchvision
from torchvision import transforms

In [2]:
# insert in utils 
def accuracy_fn(pred, y):
    return (pred.argmax(dim=1) == y).float().mean()

In [3]:
# insert in utils 

def report(loss, pred, y):
    print(f'{loss:.2f}, {accuracy_fn(preds, yb):.2f}')

In [4]:
transform = transforms.Compose([transforms.ToTensor()])
train_dataset = torchvision.datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)

In [6]:
from torch.utils.data import DataLoader, default_collate

In [7]:
def dataloader(train_dataset, test_dataset, batch_size):
    return (
        DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True, collate_fn=default_collate), 
        DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False, collate_fn=default_collate),
        train_dataset.classes
    )

In [8]:
BATCH_SIZE = 64
train_dataloader, test_dataloader, class_names = dataloader(train_dataset, test_dataset, BATCH_SIZE)

In [9]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [10]:
from conv import *
cnn = nn.Sequential(
    conv(1, 4),
    conv(4, 8),
    conv(8, 16),
    conv(16, 16),
    conv(16, 10, act=False),
    nn.Flatten()
)

In [11]:
lr = 0.4

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params=cnn.parameters(), lr=lr)

In [12]:
# insert in train 

def fit(epochs, model, train_dataloader, test_dataloader, loss_fn, optimizer, device):
    for epoch in range(epochs):
        model.to(device)
        model.train()
        for X, y in train_dataloader:
            X, y = X.to(device), y.to(device)
            loss = loss_fn(model(X), y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        model.eval()
        with torch.inference_mode():
            tot_loss, tot_acc, count = 0.,0.,0
            for X, y in test_dataloader:
                X, y = X.to(device), y.to(device)
                pred = model(X)
                n = len(X)
                count += n
                tot_loss += loss_fn(pred, y).item()*n
                tot_acc += accuracy_fn(pred, y).item()*n
        print("--------------------")
        print(count)
        print(tot_loss)
        print(tot_acc)
        print("--------------------")
        print(epoch, tot_loss/count, tot_acc/count)
        print("--------------------")
    return tot_loss/count, tot_acc/count

In [13]:
fit(epochs=3, model=cnn, loss_fn=loss_fn, optimizer=optimizer, train_dataloader=train_dataloader, test_dataloader=test_dataloader, device=device)

--------------------
10000
5402.981855869293
7994.0
--------------------
0 0.5402981855869293 0.7994
--------------------
--------------------
10000
4829.3821268081665
8239.0
--------------------
1 0.48293821268081666 0.8239
--------------------
--------------------
10000
5477.206356525421
7935.0
--------------------
2 0.5477206356525421 0.7935
--------------------


(0.5477206356525421, 0.7935)

In [14]:
def accuracy_fn(pred, y):
    return (pred.argmax(dim=1) == y).sum().item() / len(pred)

In [15]:
from tqdm import tqdm
def fit_2(epochs, model, train_dataloader, test_dataloader, loss_fn, optimizer, device):
    model.to(device)
    
    train_count, test_count = 0, 0
    train_loss, train_acc, test_loss, test_acc = 0, 0, 0, 0
    
    for epoch in tqdm(range(epochs)):
        model.train()
        for X, y in train_dataloader:
            X, y = X.to(device), y.to(device)
            y_pred = model(X)
            loss = loss_fn(y_pred, y) 
            train_loss += loss
            train_acc += accuracy_fn(y_pred, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        model.eval()
        with torch.inference_mode():
            for X, y in test_dataloader:
                X, y = X.to(device), y.to(device)
                test_pred = model(X)
                test_loss += loss_fn(test_pred, y)
                test_acc += accuracy_fn(pred=test_pred, y=y)
        
        train_count = len(train_dataloader)*(epoch+1)
        test_count = len(test_dataloader)*(epoch+1)
        
        print(f"| Epoch: {epoch} | Train loss: {train_loss / train_count } | Train accuracy: {train_acc / train_count} | Test loss: {test_loss / test_count} | Test accuracy: {test_acc / test_count} |")
    return (
        train_loss.item() / train_count,
        train_acc / train_count,
        test_loss.item() / test_count,
        test_acc / test_count
    )

In [16]:
fit_2(epochs=3, model=cnn, loss_fn=loss_fn, optimizer=optimizer, train_dataloader=train_dataloader, test_dataloader=test_dataloader, device=device)

 33%|███▎      | 1/3 [00:07<00:14,  7.28s/it]

| Epoch: 0 | Train loss: 0.43925905227661133 | Train accuracy: 0.8366371268656716 | Test loss: 0.4506646394729614 | Test accuracy: 0.8305135350318471 |


 67%|██████▋   | 2/3 [00:14<00:07,  7.04s/it]

| Epoch: 1 | Train loss: 0.42257019877433777 | Train accuracy: 0.8426255996801706 | Test loss: 0.4428204596042633 | Test accuracy: 0.834593949044586 |


100%|██████████| 3/3 [00:21<00:00,  7.13s/it]

| Epoch: 2 | Train loss: 0.4139711856842041 | Train accuracy: 0.8455545931058991 | Test loss: 0.4388471245765686 | Test accuracy: 0.8383094479830149 |


(0.41397118585365583,
 0.8455545931058991,
 0.43884711457918657,
 0.8383094479830149)